##   Blockchain Based Crypto Currency ( Pay-to-PubkeyHash (P2PKH) Version)
### The Assignment is at the end of this file, it's due 21 / 4 / 2019, any late submit won't be accepted
###   __Author__: Mahmoud Matrawy
### __Revised By__: Doha Abd EL-Fattah

 The following Assignment was created to simulate Bitcoin Transaction Verification using  Pay-to-PubkeyHash (P2PKH) scheme, check __section 4.3.1__ in the developer guide for more detials https://www.lopp.net/pdf/Bitcoin_Developer_Reference.pdf <br>
 
__ Mathematics & Computer Science Dept, Faculty of Science, Alexandria University__ <br><br>
### The raw Version From this notebook, is MaTCoinPay2PubKey.py, you will find it in the github ripo<br>

### What's new in Pay-to-PubkeyHash (P2PKH)verification scheme?
In a later version __Pay-to-address__, we put in the transaction input and output, the source address and destination address respectively, but this leaves us with a critical problem, __how the miner know if a specific user claim that he owns the coin in the transaction__, the address is not enough, anyone can claim he owns this address, we need something more challengeable to prove that a user owns a coin in a specific transaction, here come  __Pay-to-PubkeyHash__,it states that in order a user to claim he owns a coin in a specific transaction, he should able to solve a problem that is only that a user can claim the ownership of a coin in a transaction if he owns the private key corresponding to the address (public key)<br>
when creating the new transaction he provide two things's the public key (address) that exists in the __referanced transaction__ and a digital sign (this is done by __signing__ the hash of the transaction with his private key), so now any miner can verify that ownership, by decryption of sign by the public key (address) and compare the resulting hash, if the hash is equal he is the owner.<br>
#### Transaction new input's and output
Transaction Output: OP_DUP OP_HASH160 < pubKeyHash > OP_EQUALVERIFY OP_CHECKSIG <br>
Transaction Input: < sig > < pubKey > <br>
#### What the Miner to verify the claim of a user that he owns a coin in a specific transaction, the miner concatenate both input (in the new transaction) with an output (in the referenced transaction) 
#### Concatenated data: < sig > < pubKey > OP_DUP OP_HASH160 < pubKeyHash > OP_EQUALVERIFY OP_CHECKSIG <br>
Do you remember those? we created a solved an example during the section, during this project, you are going to implement functions on how to deal with such a script & verify if it's true or not, this done through the stack!
![Merkle](Pic/script.png)

### Let's Start :D 
### Adding the Imports & Transaction class, nothing new here
### Please note that, when comparing that code with the previous code (P2Address), you will notice that I removed a lot of function's because here we will focus on implementing the Pay-to-PubkeyHash (P2PKH) scheme<br>

In [43]:
from Crypto.Hash import SHA256, RIPEMD
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5
import pprint
from datetime import datetime

In [44]:
class Transaction:

    # class constructor, remember each transaction contain array of inputs and outputs
    def __init__(self, input, output):
        self.input = input  # []
        self.output = output

    # calculate the double hash of the transaction data
    def calcTransactionHash(self):
        s = b""  # this define empty array of bytes, b stand for byte!
        for x in self.input:
            s = s + bytes.fromhex(x['srcHash']) + bytes(x['index']) + bytes.fromhex(x['srcAddress'])  # concat inputs
        for y in self.output:
            s = s + bytes(y['amount']) + (y['DistAddress'].encode())  # concat outputs
            return SHA256.new(
            SHA256.new(s).digest())
        return SHA256.new(
            SHA256.new(s).digest()).digest()  # .digest(), return hash as array of bytes

    # Get amount of coin for specific output index
    def getIndexAmount(self, index):
        return (self.output[index])['amount']

    # Print the transaction data
    def printTransactionData(self):
        pprint.pprint({'input ': self.input})
        pprint.pprint({'output': self.output})

    # Get data of transaction for further print, used to print block with transaction
    def getPrintData(self):
        return {'Transaction Hash': self.calcTransactionHash().hexdigest(),
                'TransactionData': {'input': self.input, 'output': self.output}}


## Block class, also nothing new here


In [45]:
class Block:
    def __init__(self, nonce, tstamp, transactionList, prevhash, target):
        self.nonce = nonce
        self.tstamp = tstamp
        self.transactionList = transactionList
        self.transactionNumber = len(transactionList)
        self.calcMerkleTree()
        self.prevhash = prevhash
        self.target = target
        self.hash = ''

    # Calculate a hash of a block
    def calcHash(self):
        # concatenating all the data for calculate hash
        blockString = (bytes.fromhex(self.prevhash)
                       + bytes.fromhex(self.merkleTree)
                       + self.tstamp.encode()
                       + bytes(self.target)
                       + bytes(self.nonce)
                       + bytes(self.transactionNumber))  # remeber that out goal to create array of bytes

        self.hash = SHA256.new(blockString).hexdigest()
        return self.hash

    # Calculate merkle tree of transactions in a block
    def calcMerkleTree(self):
        hashList = [tran.calcTransactionHash().digest() for tran in
                    self.transactionList]  # hash list contain hash's of all transaction in block
        self.merkleTree = self.merkle(hashList).hex()  # .hex() convert array of bytes to string of hex!

    def merkle(self, hashList):
        if len(hashList) == 1:
            return hashList[0]
        newHashList = []
        for i in range(0, len(hashList) - 1, 2):
            newHashList.append(self.HashPair(hashList[i], hashList[i + 1]))
        if len(hashList) % 2 == 1:  # odd, hash last item twice
            newHashList.append(self(hashList[-1], hashList[-1]))
        return self.merkle(newHashList)

    def HashPair(self, a, b):
        return SHA256.new(SHA256.new(a + b).digest()).digest()

    def printBlockData(self):
        dict = {
            "BlockHash": self.hash,
            'MerkleTreeHash': self.merkleTree,
            'prevHash': self.prevhash,
            'Target': self.target,
            'Timestamp': self.tstamp,
            'TransactionNumbers': self.transactionNumber,
            'Transactions List': [tran.getPrintData() for tran in self.transactionList],
        }
        pprint.pprint(dict)



### Blockchain Class
### Here we will enconter some changes
### in the function calcGensisBlock(self, listOfAddress, peers):
__distAddress = "OP_DUP OP_HASH160 "+peer.address+" OP_EQUALVERIFY OP_CHECKSIG"__ <br>
in the output of the transaction, i didn't put only the address, but instead put a script, because later when he wan'ts to use this transaction he should provide the signature + publickey.

In [46]:
class Blockchain:

    def __init__(self):
        self.blockchain = []
        self.peers = []

    # Gensis block, is the initial block that contain initial transaction !
    def calcGensisBlock(self, listOfAddress, peers):
        
        listOfTransaction = []
        for address,peer in zip(listOfAddress,peers): # here i loop over two list, technolgya b2a ya gd3 :D
            #*new variable
            distAddress = "OP_DUP OP_HASH160 "+peer.address.hex()+" OP_EQUALVERIFY OP_CHECKSIG"

            listOfTransaction.append(
                Transaction([{'srcHash': '00', 'index': 0, 'srcAddress': '00'}, ]  # 00 equivalent to null
                            ,
                            [{'amount': address['amount'], 'DistAddress': distAddress}, ]))

        gensisBlock = Block(0, '2012/8/5', listOfTransaction, '00', 0)  # first block created was in  2012
        gensisBlock.calcHash()
        self.blockchain.append(gensisBlock)  # Add the gensis block to blockchain
        i = 0
        for i in range(0, len(peers)):  # Update the utxo(unspend transaction output)
            peers[i].utxo.append({'transaction': listOfTransaction[i], 'index': 0})
            peers[i].CalcAmount()
            i = i + 1

    # function to print all the block data in the blockchain
    def printBlockData(self):
        for block in self.blockchain:  # loop thorugh blockchain and print each block data
            block.printBlockData()

    # function to print all the transaction in the blockchain without block data
    def printAllTransaction(self):
        list = []
        for block in self.blockchain:
            for transaction in block.transactionList:
                fromm = transaction.input[0]["srcAddress"]  # to know transaction sender
                to = ', '.join([output['DistAddress'] for output in
                                transaction.output])  # to know the receiver
                amount = ', '.join(
                    [str(output['amount']) for output in transaction.output])  # amount of money send to each receiver
                list.append({"Transaction Hash: ": transaction.calcTransactionHash().hexdigest(),
                             "From Address ": fromm ,
                             "To Address": to, "Amount": amount})
        return list


### Actor Class
### Here we will enconter change's in the def SendCoin(self, to, Amount)
#### singer = PKCS1_v1_5.new(self.rsaKey), used to sign the referanced transaction that is used as input, to claim that he own the coins in the referanced transaction.
#### srcAdd = singer.sign(tran.calcTransactionHash()).hex() + " "+self.rsaKey.publickey().exportKey().hex(), here he sign the referenced transaction then add a space then add his public key ( that later will converted to address! )

In [47]:
class Actor:
    ''' parameters
    fileName : contain  the path to pem of the user
    blockchain : contain the main object of blockchain '''

    def __init__(self, fileName, blockchain):
        f = open(fileName, 'r')  # open the file
        self.rsaKey = RSA.importKey(f.read())  # using RSA module to import kye from pem
        self.address = RIPEMD.new(self.rsaKey.publickey().exportKey()).digest()  # address is hash 160 of public key
        self.amount = 0  # init amount of coin to 0
        self.blockchain = blockchain  # assign blockchain object
        self.utxo = []  # utxo is empty now
        self.publicKey = self.rsaKey.publickey().exportKey()
        f.close()  # close pem file

    # function to calculate amount of money for a user
    def CalcAmount(self):
        self.amount = 0
        for raw in self.utxo:  # loop through the user utxo and calc amount
            self.amount += raw['transaction'].getIndexAmount(raw['index'])

    # send coin function
    ''' parameters
        to : object of the receiver (instance from Actor)
        Amount : amount of coin you want to send to receiver '''    
    def SendCoin(self, to, Amount):
        if len(self.utxo) == 0:  # check the utxo if empty, then he don't have coin!
            print('Address: ', self.address.hex(), " do not have coins")
            return None
        listOfInput = []
        count = 0  # counter to know value for every input transaction
        # 3la 7sb b2a, mmoken ast5dm aktr mn input, 3shan msln lw input wa7d  mmoken mekfesh el coin el feh
        
        #* New part*
        singer = PKCS1_v1_5.new(self.rsaKey)
        #**
        
        for raw in self.utxo:
            tran = raw['transaction']
            count += tran.getIndexAmount(raw['index'])
            
            #* New part
            srcAdd = singer.sign(tran.calcTransactionHash()).hex() + " "+self.rsaKey.publickey().exportKey().hex()
            #**
            
            listOfInput.append(
                {'srcHash': tran.calcTransactionHash().digest().hex(), 'index': raw['index'], 'srcAddress': srcAdd})
            if count >= Amount:  # here break because count is grater than amount to be send
                break
        if (count < Amount):  # here the coin is not enough
            print('Address: ', self.address, " coins is not enough")
            return None

        diff = count - Amount  # calculate the diff
        #* New part
        distRec = "OP_DUP OP_HASH160 "+to.address.hex()+" OP_EQUALVERIFY OP_CHECKSIG"
        #**
        
        newTransaction = Transaction(listOfInput, [
            {"amount": (Amount), "DistAddress": distRec}])  # define output of transaction

        self.utxo = self.utxo[len(listOfInput)::]  # remove used transaction from utxo
        if count != Amount:
            #* New part
            distSend = "OP_DUP OP_HASH160 "+self.address.hex()+" OP_EQUALVERIFY OP_CHECKSIG"
            #**
            
            newTransaction.output.append({'amount': diff, 'DistAddress':distSend})
            self.utxo.append({'transaction': newTransaction, 'index': 1})  # hard coded :(

        to.utxo.append({'transaction': newTransaction, 'index': 0})

        # here update the balance for each send and receiver
        to.CalcAmount()
        self.CalcAmount()
        return newTransaction  # when done return the new transaction for mining later
    def getUserData(self):
        self.CalcAmount()
        return {"Public key: ": (self.rsaKey.publickey().exportKey()[27::])[:-25].hex(),
                "Private key: ": (self.rsaKey.exportKey()[32::])[:-30].hex(),
                "Coin address:": self.address.hex(),
                "Amount ": self.amount,
                "Transaction UTXO": [tran['transaction'].calcTransactionHash().digest().hex() for tran in self.utxo]}

In [48]:
# Lets check the changes that we made 

mainBlockchain = Blockchain()
Alice = Actor('alice.pem', mainBlockchain)
Bob = Actor('bob.pem', mainBlockchain)

listOfGensisTransaction = [{'amount': 50, 'address': Alice.address.hex()},
                           {'amount': 150, 'address': Bob.address.hex()}]
mainBlockchain.calcGensisBlock(listOfGensisTransaction, [Alice, Bob])
print(Alice.amount)

50


In [49]:
#Lets check the new transaction data
TranSend = Alice.SendCoin(Bob, 10)
print(TranSend.printTransactionData())

{'input ': [{'index': 0,
             'srcAddress': '870153834c8895302f615c1c53356f1c1f2af90762557fb5490d68ddfd0c968a3d78a0110ff28ce134950745f6fa4feb471b243bc63e137d043bf6649aaec72f4d719dab43df327614ae99d41ce9dcba666b762dce55f640b592b388e6c531fe7d6a136cc082762b129a9b35d8b95086271f3b9553030325a5c213bf03e1806dc5d4ec5295d9d415a5f81ee31cd2a65b0e141557840d0a82af64447a72b19661d1a4ff7a96478de564cd445c0237b10e49f61acb93b88ca8fd43b2f1423094a972ff70806aafff658aa61428081fe78f0ef9e28f4412273e05ed3a209115c0f94bc6e87ee71ccb9bf1cc9faae4420ffa97604bd4c63ba3c46b946023ab9fd440 '
                           '2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d494942496a414e42676b71686b6947397730424151454641414f43415138414d49494243674b434151454171796f776959365948664a727a58496970364c570a6f78564a77485963584d5245424b305051346f71625a4970775772416c624e6d30684f2f6d56755858537655667445617a734f31784e6341373279764d734c550a4e4362427655454e4f46514568755841733568714c412b374d3830525061694264702f436c50614f6761636d66

### Let's interpret the previous output
#### in the input part, check the src address: we have a long string of hexadecimal that represents the signature of the previous transaction followed by space then the public key of the sender
#### in the output part DistAddress =  OP_DUP OP_HASH160 78c05a4f9444faec2de6d65227dcabc734e0d163 OP_EQUALVERIFY OP_CHECKSIG
#### Rather than putting the direct address, we now put a script, later you should how to process this script 🙄🙄


# Assignment Part
## Mark Weight: 15
### def Mining(self, currentTarget):
for tran in self.pendingTransaction:<br>
&nbsp;&nbsp;&nbsp;if not __self.Verify(tran)__:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return False <br>
### Here you loop on the pending transaction to check the verification of every transaction, your goal is to implement this function 💪, check the following code

In [50]:
class Miner(Actor):

    def __init__(self, fileName, blockchain):
        Actor.__init__(self, fileName, blockchain)
        self.pendingTransaction = []  # list of pending transaction to be mined later
        self.reward = 0

    # Function to return balance for specific address, another way to calculate balance
    ''' parameters
        currentTarget : define the difficulty of mathematical puzzle '''

    def Mining(self, currentTarget):
        currentTime = datetime.now().strftime("%m/%d/%Y%H:%M:%S")  # get current time!
        
        # create new object from block for mining
        minedBlock = Block(0, currentTime, self.pendingTransaction.copy(),
                           self.getTopBlock().hash, currentTarget)

        # adding the reward transaction!, fees for mining
        # src address 0000 it's a mining fee
        
        #*new part
        distAddress = "OP_DUP OP_HASH160 "+self.address.hex()+" OP_EQUALVERIFY OP_CHECKSIG"
        #* 
        
        for tran in self.pendingTransaction:
            if not self.Verify(tran):
                return False
            
        minedBlock.transactionList.append(
            Transaction([{'srcHash': '0000', 'index': 0, 'srcAddress': '0000'}, ]
                        ,
                        [{'amount': self.reward, 'DistAddress': distAddress}, ]))

        '''here in the loop every time we calculate the hash of the block and compare the leading chars 
         in the hash if they are zeros or not according to the target, for example, if target is 2, then
          we compare the leading two char  if they are 00 or not, it yes mining done, if not, increment 
          the nonce var and then calculate the hash again, and you loop until you satisfies the condition '''

        while minedBlock.hash[:currentTarget] != "0" * currentTarget:  # mining condition
            minedBlock.nonce += 1
            minedBlock.hash = minedBlock.calcHash()

        print("Mined Successfully")
        self.pendingTransaction.clear()
        self.blockchain.blockchain.append(minedBlock)

    # to Do you have to replace None with correct Solution ^^
    # first we need to check that the referanced transction exisit in the blockchain
    #if found then check the script..
    def Verify(self,Tran):
        for input in None:
            for block in None:
                for prevTran in None:
                        if input['srcHash'] == prevTran.calcTransactionHash().hexdigest():
                            if not self.checkScript(input, prevTran.output[input['index']], prevTran.calcTransactionHash()):
                                return False
        return True

    '''
    the checkScript function, a function where it checks the correctness of a script, here we do the stack table 
        CurrentTransactionInput: input part in the current transaction, we need it to get the signature & public key
        prevTran: output part in the referenced transaction, we need it to get the script part :D 
        TranHash: the hash of the referenced transaction
    '''
    
    def checkScript(self,CurrentTransactionInput,prevTran,TranHash):
        
        #concate the signture and address with the script part
        concate = CurrentTransactionInput['srcAddress']+" "+prevTran['DistAddress']
        
        #we need to split the data into a list
        #remember that it's separated by a space, aktr mn kda agy a3mel el split ana 😂
        #concate = (here we put the function of split)
        
        #concate data = < sig > < pubKey > OP_DUP OP_HASH160 < pubKeyHash > OP_EQUALVERIFY OP_CHECKSIG
        l = []
        
        operation = ['OP_DUP','OP_HASH160','OP_EQUALVERIFY','OP_CHECKSIG']
        
        for data in concate:
            if data not in operation:
                l.append(data)
            if data == "OP_DUP":
                top = l.pop()
                # fill here, what we do after we do a pop ?
            if data == "OP_HASH160":
                top = l.pop()
                #fill here, remeber, that the public key ia a string of hex, convert it back to array of byte
                #by bytes.fromhex() , then hash then push to stack
            if data == 'OP_EQUALVERIFY':
                print("")
                #fill here, what we do here?
            if data == 'OP_CHECKSIG':
                pub_Key = l.pop()
                sig = l.pop()
                pub_Key = RSA.importKey(bytes.fromhex(pub_Key))
                #here you have Sig, pub_Key, you need to verify the signature, if it's correct return true
        
        #if the for loop reach it's end and didn't return true, then the verification scheme failed
        return False

    def getTopBlock(self):
        return self.blockchain.blockchain[-1]

In [51]:
#let's test

mainBlockchain = Blockchain()
Alice = Actor('alice.pem', mainBlockchain)
Bob = Actor('bob.pem', mainBlockchain)
Mahmoud = Miner('mahmoud.pem', mainBlockchain)


listOfGensisTransaction = [{'amount': 50, 'address': Alice.address.hex()},
                           {'amount': 150, 'address': Bob.address.hex()}]
mainBlockchain.calcGensisBlock(listOfGensisTransaction, [Alice, Bob])
print(Alice.amount)
Mahmoud.reward = 2 # setting mining reward
newTransaction = Alice.SendCoin(Bob, 10)
Mahmoud.pendingTransaction.append(newTransaction)


50


In [52]:
Mahmoud.Mining(2)

TypeError: 'NoneType' object is not iterable

### The error because you have to replace the None with the correct solution,
### Expected output that you should get __Mined Successfully__
### Good Luck, if you have any query, don't hasitate to message me